In [1]:
import pandas as pd
import json

In [11]:
skoly = pd.read_csv("schools_with_villages.csv")[["village", "school", "num_students"]]

In [12]:
skoly

,village,school,num_students
0,Abovce,ZŠ s MŠ - Abovce,34.0
1,Abovce,Materská škola pri ZŠ - Abovce,14.0
2,Abrahám,"Materská škola - Abrahám, Hviezdoslavova",39.0
3,Abrahám,"ZŠ - Abrahám, Školská",130.0
4,Abrahámovce,Materská škola - Abrahámovce,10.0
...,...,...,...
6566,Žlkovce,ZŠ - Žlkovce,29.0
6567,Župčany,ZŠ s MŠ - Župčany,55.0
6568,Župčany,EP ZŠ s MŠ (MŠ) - Župčany,39.0
6569,Župkov,Materská škola pri ZŠ - Župkov,25.0


In [20]:
obce = pd.read_csv("https://raw.githubusercontent.com/cedeerwe/korona-data-uderka/master/pairs.csv", usecols=[1,3], names=["village", "district"])

In [88]:
obce_map = {obec["village"]: obec["district"] for i, obec in obce.iterrows()}

In [32]:
len(obce)

4208

In [27]:
with open("../obyvatelia/obce_obyvatelia_fixed.json", "r") as f:
    obyvatelia = json.loads(f.read())
obyvatelia = pd.DataFrame([{"village": obec["obec"], "population": obec["pocet_obyvatelov_2013"]} for obec in obyvatelia])

In [33]:
len(obyvatelia)

2927

In [39]:
obyvatelia_with_district = obyvatelia.merge(obce, on="village")

In [43]:
filtered_obce = obyvatelia_with_district[obyvatelia_with_district['population'] >= 5000]

In [104]:
obyvatelia_per_district = {
    district: obyvatelia_with_district[
        obyvatelia_with_district['district'] == district
    ]['population'].sum() 
    for district in set(obce_map.values())}

In [126]:
obce_so_skolami = set(skoly['village'])

def skola_v_obci(obec):
    if obec == "Šaštín - Stráže":
        return "Základná umelecká škola - Šaštín-Stráže, J.Hollého"
    elif obec in obce_so_skolami:
        obec_df = skoly[skoly["village"] == obec]
        skoly_obec_studenti = obec_df[obec_df['num_students'].notnull()]
        if len(skoly_obec_studenti) == 0:
            return obec_df["school"].values[0]
        else:
            return skoly_obec_studenti.iloc[skoly_obec_studenti['num_students'].argmax()]['school']
    else:
        if "Bratislava" in obec or "Košice" in obec:
            return "Nechávame na uvážení okresu"
        else:
            print("Chyba, nemame skolu pre: ", obec)

from collections import defaultdict      
result = defaultdict(list)
            
for i, obec in filtered_obce.iterrows():
    district = obce_map[obec["village"]]
    
    result_for_obec = {
        "Obec": obec["village"],
        "Navrhované odberové miesto": skola_v_obci(obec["village"]),
        "Počet obyvateľov": obec["population"],
        # "Navrhovaný počet odberových jednotiek": "{:.2f}".format(obec["population"] / 5000)
    }
    result[district].append(result_for_obec)

result2 = {}
for district, district_obce in result.items():
    pocet_odberovych_miest = obyvatelia_per_district[district] / 5000
    real_pocet_odberovych_miest = 0
    obyvatelia_v_odberovych = sum(obec['Počet obyvateľov'] for obec in district_obce)
    odberove = []
    for obec in district_obce:
        odberovych = round(obec["Počet obyvateľov"] / obyvatelia_v_odberovych * pocet_odberovych_miest)
        real_pocet_odberovych_miest += odberovych
        obec["Navrhovaný počet odberových jednotiek pre obec"] = odberovych
        odberove.append(obec)
    result2[district] = {
        "Navrhovaný počet odberových miest pre okres": real_pocet_odberovych_miest,
        "Odberové miesta": odberove
    }

# to csv
csv_rows = []
for district, value in result2.items():
    csv_rows.append({
        "Okres": district,
        "Počet obyvateľov v okrese": obyvatelia_per_district[district],
        "Navrhovaný počet odberových miest pre okres": value["Navrhovaný počet odberových miest pre okres"],
    })
    for miesto in value['Odberové miesta']:
        csv_rows.append(miesto)

pd.DataFrame(csv_rows).to_csv("odberove_miesta.csv")       

In [122]:
import yaml
with open("odberove_miesta.txt", "w") as f:
    yaml.dump(result2, f)

In [78]:
set(filtered_obce['village']) - set(result.keys())

set()

In [81]:
len(set(filtered_obce['village']))


144